# Intro

In this notebook, I try to make some random predictions.

In [43]:
from os.path import join as join_paths
from typing import Any

from numpy import ndarray
from pandas import read_csv, DataFrame, concat
from sklearn.dummy import DummyClassifier

from calculateEvaluationCCC import ccc

In [16]:

default_path: str = "/Users/leonardoalchieri/Datasets/OMGEmotionChallenge"
validation_data = read_csv(join_paths(default_path, "omg_ValidationVideos.csv"))
# the x is not necessary: instead of using some random values, I will take the "start" column
x: ndarray = validation_data.loc[:, "start"].values
y_valence: ndarray = validation_data.loc[:, "valence"].values
y_arousal: ndarray = validation_data.loc[:, "arousal"].values


In [46]:
seeds: list[int] = [23, 43, 23, 45, 305353, 1998, 232423, 234540294, 2029004, 1, 2, 3, 1234]
cccs: list[DataFrame] = list() 
for seed in seeds:
    ccc_valences: dict[str, float] = dict()
    ccc_arousals: dict[str, float] = dict()
    strategies: list[str] = ["stratified", "most_frequent", "constant", "uniform", "prior"]
    strategies_args: dict[str, dict[str, Any]] = {'constant': {'constant': 0}}
    for strategy in strategies:
        classifier_valence = DummyClassifier(strategy=strategy, 
                                            random_state=seed, 
                                            **strategies_args.get(strategy, dict()))
        classifier_arousal = DummyClassifier(strategy=strategy, 
                                            random_state=seed, 
                                            **strategies_args.get(strategy, dict()))
        
        classifier_valence.fit(X=x, y=y_valence)
        classifier_arousal.fit(X=x, y=y_arousal)
        
        predictions_valence: ndarray = classifier_valence.predict(x)
        # print(f"{predictions_valence=}")
        predictions_arousal: ndarray = classifier_arousal.predict(x)
        # print(f"{predictions_arousal=}")
        
        ccc_valence, rho_valence = ccc(y_true=y_valence, 
                                    y_pred=predictions_valence)
        ccc_arousal, rho_arousal = ccc(y_true=y_arousal, 
                                    y_pred=predictions_arousal)
        
        ccc_valences[strategy] = ccc_valence
        ccc_arousals[strategy] = ccc_arousal
    cccs.append(DataFrame([ccc_valences, ccc_arousals], index=['valence', 'arousal']))

/Users/leonardoalchieri/miniconda3/envs/torch_latest/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/leonardoalchieri/miniconda3/envs/torch_latest/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/leonardoalchieri/miniconda3/envs/torch_latest/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/leonardoalchieri/miniconda3/envs/torch_latest/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.Con

In [47]:
display(concat(cccs, keys=strategies, axis=0).groupby(axis=0, level=1).mean())

,stratified,most_frequent,constant,uniform,prior
arousal,0.027090,NaN,NaN,0.027584,NaN
valence,-0.010433,NaN,NaN,0.002714,NaN
